In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [13]:
pd.read_sql(
    '''
    select * from(
    SELECT * FROM boston
    
    )
    ''',
    con,
).shape

(506, 14)

In [6]:
tasks = set()

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
tasks.add(1)
pd.read_sql(
    '''
    SELECT 'CRIM' as column_name, COUNT(CRIM) as null_count FROM boston
    WHERE CRIM is Null
    UNION
    SELECT 'ZN', COUNT(ZN) FROM boston
    WHERE ZN is Null
    UNION
    SELECT 'INDUS', COUNT(INDUS) FROM boston
    WHERE INDUS is Null
    UNION
    SELECT 'CHAS', COUNT(CHAS) FROM boston
    WHERE CHAS is Null
    UNION
    SELECT 'NOX', COUNT(NOX) FROM boston
    WHERE NOX is Null
    ''',
    con,
)

,column_name,null_count
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks.add(2)
pd.read_sql(
    '''
    SELECT 'CRIM' as column_name, COUNT(DISTINCT CRIM) as unique_count FROM boston
    UNION
    SELECT 'ZN', COUNT(DISTINCT ZN) FROM boston
    UNION
    SELECT 'INDUS', COUNT(DISTINCT INDUS) FROM boston
    UNION
    SELECT 'CHAS', COUNT(DISTINCT CHAS) FROM boston
    UNION
    SELECT 'NOX', COUNT(DISTINCT NOX) FROM boston
    ''',
    con,
)

,column_name,unique_count
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
tasks.add(3)
pd.read_sql(
    '''
    SELECT * FROM (
        SELECT 'CRIM' AS column_name, 
        MIN(CRIM) AS min,    
        (SELECT AVG(med) AS median FROM (
            SELECT MAX(CRIM) AS med FROM (
                SELECT * FROM boston 
                ORDER BY CRIM 
                LIMIT (SELECT ROUND(COUNT(CRIM)/2.0) FROM boston)
            )
            UNION
            SELECT MIN(CRIM) AS med FROM (
                SELECT * FROM boston 
                ORDER BY CRIM DESC
                LIMIT (SELECT ROUND(COUNT(CRIM)/2.0) FROM boston)
            )
        )) AS median FROM boston

        UNION 

        SELECT 'ZN' AS column_name, 
        MIN(ZN) AS min,    
        (SELECT AVG(med) AS median FROM (
            SELECT MAX(ZN) AS med FROM (
                SELECT * FROM boston 
                ORDER BY ZN 
                LIMIT (SELECT ROUND(COUNT(ZN)/2.0) FROM boston)
            )
            UNION
            SELECT MIN(ZN) AS med FROM (
                SELECT * FROM boston 
                ORDER BY ZN DESC
                LIMIT (SELECT ROUND(COUNT(ZN)/2.0) FROM boston)
            )
        )) AS median FROM boston

        UNION

        SELECT 'INDUS' AS column_name, 
        MIN(INDUS) AS min,    
        (SELECT AVG(med) AS median FROM (
            SELECT MAX(INDUS) AS med FROM (
                SELECT * FROM boston 
                ORDER BY INDUS 
                LIMIT (SELECT ROUND(COUNT(INDUS)/2.0) FROM boston)
            )
            UNION
            SELECT MIN(INDUS) AS med FROM (
                SELECT * FROM boston 
                ORDER BY INDUS DESC
                LIMIT (SELECT ROUND(COUNT(INDUS)/2.0) FROM boston)
            )
        )) AS median FROM boston

        UNION 

        SELECT 'CHAS' AS column_name, 
        MIN(CHAS) AS min,    
        (SELECT AVG(med) AS median FROM (
            SELECT MAX(CHAS) AS med FROM (
                SELECT * FROM boston 
                ORDER BY CHAS 
                LIMIT (SELECT ROUND(COUNT(CHAS)/2.0) FROM boston)
            )
            UNION
            SELECT MIN(CHAS) AS med FROM (
                SELECT * FROM boston 
                ORDER BY CHAS DESC
                LIMIT (SELECT ROUND(COUNT(CHAS)/2.0) FROM boston)
            )
        )) AS median FROM boston

        UNION

        SELECT 'NOX' AS column_name, 
        MIN(NOX) AS min,    
        (SELECT AVG(med) AS median FROM (
            SELECT MAX(NOX) AS med FROM (
                SELECT * FROM boston 
                ORDER BY NOX 
                LIMIT (SELECT ROUND(COUNT(NOX)/2.0) FROM boston)
            )
            UNION
            SELECT MIN(NOX) AS med FROM (
                SELECT * FROM boston 
                ORDER BY NOX DESC
                LIMIT (SELECT ROUND(COUNT(NOX)/2.0) FROM boston)
            )
        )) AS median FROM boston
    ) 
    WHERE min = median
    
    ''',
    con,
)

,column_name,min,median
0,CHAS,0.0,0.0
1,ZN,0.0,0.0


*Большая часть значений по колонкам CHAS и ZN заполнена минимальными значениями. Т.е. большая часть районов не граничит с рекой, и большая часть районов имеет долю земель под жилую застройку зонированных под участки площадью более 25 000 кв футов равную 0*

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks.add(4)
pd.read_sql(
    '''
    WITH EXPENCIVE AS (
        SELECT AVG(RM) FROM BOSTON where MEDV = (select MAX(MEDV) from boston)
    )
    
    
    SELECT houses_count,  avg_rm, max_price_avg_rm, max_price_avg_rm - avg_rm AS diff FROM 
    (
        SELECT '25' as houses_count, AVG(RM) as avg_rm, (SELECT * FROM EXPENCIVE) as max_price_avg_rm    
        FROM (
            SELECT RM, MEDV FROM boston
            ORDER BY MEDV
            LIMIT 25
        ) 

        UNION ALL

        SELECT '50', AVG(RM), (SELECT * FROM EXPENCIVE) FROM (
            SELECT RM, MEDV FROM boston
            ORDER BY MEDV
            LIMIT 50
        ) 

        UNION ALL

        SELECT '100', AVG(RM), (SELECT * FROM EXPENCIVE) FROM (
            SELECT RM, MEDV FROM boston
            ORDER BY MEDV
            LIMIT 100
        ) 

        UNION ALL

        SELECT '200', AVG(RM), (SELECT * FROM EXPENCIVE) FROM (
            SELECT RM, MEDV FROM boston
            ORDER BY MEDV
            LIMIT 200
        ) 

        UNION ALL

        SELECT '300', AVG(RM), (SELECT * FROM EXPENCIVE) FROM (
            SELECT RM, MEDV FROM boston
            ORDER BY MEDV
            LIMIT 300
        )         
    
    )    

    ''',
    con,
)

,houses_count,avg_rm,max_price_avg_rm,diff
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


*Тк с добавлением записей (от 25 до 300) с более дорогой недвижимостью, растет и среднее значение комнат в домах и разница со средним количеством комнат в самой дорогой недвижимости снижается, можно предположить, что связь между средним количестовм комнат и ценой есть*

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [15]:
tasks.add(5)
pd.read_sql(
    '''    
    SELECT * FROM(
        SELECT LSTAT, MEDV, rank() OVER(ORDER BY LSTAT) as rank FROM boston
    )    
    WHERE MEDV = (select MAX(MEDV) from boston)
   
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,6
3,2.96,50.0,8
4,2.97,50.0,9
5,3.16,50.0,15
6,3.26,50.0,17
7,3.32,50.0,18
8,3.70,50.0,26
9,3.73,50.0,27


*Основная часть дорогой недвижимости имеют низкий процент LSTAT - чуть больше 5% (последние 3 записи можно скорее отнести к выбросам)*

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [20]:
tasks.add(6)
pd.read_sql(
    '''
    SELECT CASE WHEN CHAS = 0 THEN 'не граничит' ELSE 'граничит' END AS CHAS, AVG(MEDV) FROM boston
    GROUP BY CHAS
    ''',
    con,
)

,CHAS,AVG(MEDV)
0,не граничит,22.093843
1,граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [44]:
tasks.add(7)
pd.read_sql(
    '''
    WITH MEANS AS (
        SELECT * FROM(
            SELECT CHAS, 
            AVG(CRIM) OVER (PARTITION BY CHAS) as mean_crim, 
            AVG(ZN) OVER (PARTITION BY CHAS)  as mean_zn,
            AVG(INDUS) OVER (PARTITION BY CHAS) as mean_indus, 
            AVG(NOX) OVER (PARTITION BY CHAS) as mean_nox FROM boston
        )
        GROUP BY CHAS        
    )
    
    SELECT * FROM(
        SELECT DISTINCT 'CRIM' as column, 
        (SELECT mean_crim FROM MEANS WHERE CHAS = 0) as chas_n, 
        (SELECT mean_crim FROM MEANS WHERE CHAS = 1) as chas_y 
        FROM MEANS

        UNION ALL

        SELECT DISTINCT 'ZN' as column, 
        (SELECT mean_zn FROM MEANS WHERE CHAS = 0) as chas_n, 
        (SELECT mean_zn FROM MEANS WHERE CHAS = 1) as chas_y 
        FROM MEANS

        UNION ALL

        SELECT DISTINCT 'INDUS' as column, 
        (SELECT mean_indus FROM MEANS WHERE CHAS = 0) as chas_n, 
        (SELECT mean_indus FROM MEANS WHERE CHAS = 1) as chas_y 
        FROM MEANS

        UNION ALL

        SELECT DISTINCT 'NOX' as column, 
        (SELECT mean_nox FROM MEANS WHERE CHAS = 0) as chas_n, 
        (SELECT mean_nox FROM MEANS WHERE CHAS = 1) as chas_y 
        FROM MEANS
    )        
    
    WHERE chas_y > chas_n    
    
    ''',
    con,
)

,column,chas_n,chas_y
0,INDUS,11.019193,12.719143
1,NOX,0.551817,0.593426


*В районах прилегающих к реке больше промышленных участков, тк средние значения по доле неторгового бизнеса и концентрации азота в воздухе выше*

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [60]:
tasks.add(8)
pd.read_sql(
    '''
    WITH PERCENTILE AS (
        SELECT INDUS, NOX, NTILE(10) OVER(ORDER BY INDUS, NOX) AS percentile_n FROM boston
    )
    
    SELECT '10' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 1
    UNION ALL
    SELECT '20' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 2
    UNION ALL
    SELECT '30' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 3
    UNION ALL
    SELECT '40' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 4
    UNION ALL
    SELECT '50' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 5
    UNION ALL
    SELECT '60' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 6
    UNION ALL
    SELECT '70' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 7
    UNION ALL
    SELECT '80' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 8
    UNION ALL
    SELECT '90' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 9
    UNION ALL
    SELECT '100' AS percentile, AVG(INDUS) as avg_indus, AVG(NOX) as avg_nox 
    FROM PERCENTILE 
    WHERE percentile_n = 10
    
    ''',
    con,
)

,percentile,avg_indus,avg_nox
0,10,2.016078,0.438378
1,20,3.645294,0.494129
2,30,5.246275,0.448235
3,40,6.471765,0.470839
4,50,8.256078,0.531510
5,60,10.749216,0.515176
6,70,17.245800,0.564920
7,80,18.100000,0.684880
8,90,18.307200,0.714620
9,100,21.938800,0.692860


*В целом с ростом среднего знаечния индекса INDUS растет и среднее значение индекса NOX, но в некоторых группах при этом можно видеть уменьшение среднего значения NOX*

In [62]:
if len(tasks) == 8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
